# Capstone Data Wrangling
#### Springboard Data Science Career Track
##### Tamara Monge

### I. Import the data

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df_full = pd.read_csv ('/Users/tami/Documents/Springboard/Capstone1/Parking_Citations.csv')
df_full.info()

/Users/tami/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1345053 entries, 0 to 1345052
Data columns (total 21 columns):
Citation           1345053 non-null int64
Tag                1344837 non-null object
ExpMM              1300960 non-null object
ExpYY              1345047 non-null float64
State              1345053 non-null object
Make               1343712 non-null object
Address            1345051 non-null object
ViolCode           1344668 non-null float64
Description        1345053 non-null object
ViolFine           1344668 non-null object
ViolDate           1340590 non-null object
Balance            1345053 non-null object
PenaltyDate        0 non-null float64
OpenFine           1344668 non-null object
OpenPenalty        1344668 non-null object
NoticeDate         602001 non-null object
ImportDate         1345053 non-null object
Neighborhood       206670 non-null object
PoliceDistrict     206670 non-null object
CouncilDistrict    206691 non-null float64
Location           1323450 non-nul

As shown above, the initial DataFrame was composed of 21 columns: 16 object Series (Tag, ExpMM, State, Make, Address, Description, ViolFine, ViolDate, Balance, OpenFine, OpenPenalty, NoticeDate, ImportDate, Neighborhood, PoliceDistrict, Location), 4 float64 Series (ExpYY, ViolCode, PenaltyDate, CouncilDistrict) and 1 int64 Series (Citation).

### II. Remove irrelevant cohort
The initial DataFrame contained two distinct cohorts of data: (1) all citations issued over the past two years and (2) any citations issued more than two years ago that still have an open balance due. Since this study is concerned with the former cohort, the first data wrangling step was to convert the ViolDate column to a datetime object and use it to extract the first cohort into its own DataFrame. 

In [2]:
df_full['ViolDate'] = df_full['ViolDate'][pd.notnull(df_full['ViolDate'])].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))
df = df_full[:][df_full['ViolDate'] >= '2015-09-23']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753387 entries, 5 to 1345052
Data columns (total 21 columns):
Citation           753387 non-null int64
Tag                753260 non-null object
ExpMM              725018 non-null object
ExpYY              753387 non-null float64
State              753387 non-null object
Make               752981 non-null object
Address            753387 non-null object
ViolCode           753387 non-null float64
Description        753387 non-null object
ViolFine           753387 non-null object
ViolDate           753387 non-null datetime64[ns]
Balance            753387 non-null object
PenaltyDate        0 non-null float64
OpenFine           753387 non-null object
OpenPenalty        753387 non-null object
NoticeDate         171237 non-null object
ImportDate         753387 non-null object
Neighborhood       123652 non-null object
PoliceDistrict     123652 non-null object
CouncilDistrict    123661 non-null float64
Location           743041 non-null object


The resulting DataFrame consists of 753,387 rows.

### III. Drop irrelevant columns
The next data wrangling step was to drop the irrelevant columns: ExpMM, ExpYY, PenaltyDate, NoticeDate, ImportDate, Neighborhood, PoliceDistrict, and CouncilDistrict.

In [3]:
df.drop(['ExpMM', 'ExpYY', 'PenaltyDate', 'NoticeDate', 'ImportDate', 'Neighborhood', 'PoliceDistrict', 'CouncilDistrict'], axis=1, inplace=True)

### IV. Clean the columns
Next, we needed to clean the individual columns.

#### A) Unaltered Columns
Five columns (Citation, Tag, ViolCode, Description, Address) required no cleaning.

#### B) Adding year, month, day, and hour Columns 
Using the ViolDate column of datetime objects, we saved the year, month, day, and hour as new columns in the DataFrame.

In [4]:
df['year']  = df['ViolDate'][pd.notnull(df['ViolDate'])].dt.year.astype(int)
df['month'] = df['ViolDate'][pd.notnull(df['ViolDate'])].dt.month.astype(int)
df['day']   = df['ViolDate'][pd.notnull(df['ViolDate'])].dt.day.astype(int)
df['hour']  = df['ViolDate'][pd.notnull(df['ViolDate'])].dt.hour.astype(int)

#### C) Financial Columns
Four columns (ViolFine, Balance, OpenFine, and OpenPenalty) contained financial data in string format that needed to be converted to floats. This process required 3 cleaning steps: removing null values, applying a lambda function to trim the '$' and converting the strings to floats.

In [5]:
df['ViolFine']    = df['ViolFine'][df['ViolFine'].apply(type) == str].apply(lambda x: x[1:]).astype(float) 
df['Balance']     = df['Balance'][df['Balance'].apply(type) == str].apply(lambda x: x[1:]).astype(float)   
df['OpenFine']    = df['OpenFine'][df['OpenFine'].apply(type) == str].apply(lambda x: x[1:]).astype(float)   
df['OpenPenalty'] = df['OpenPenalty'][df['OpenPenalty'].apply(type) == str].apply(lambda x: x[1:]).astype(float)   

#### D) String Columns 
Three of the columns (Address, State, and Make) contained strings with inconsistent cases that needed to be standardized. In addition, the Make column contained strings of inconsistent lengths to represent the same category (e.g., 'Hon' and 'HONDA') and thus needed the string length needed to be standardized.

In [6]:
df['Address'] = df['Address'].str.upper()
df['State'] = df['State'].str.upper()
df['Make']  = df['Make'].str.upper()
df['Make']  = df['Make'][df['Make'].apply(type) == str].apply(lambda x: x[:3]) 

#### E) Dissect the Location Column into latitude, longitude, street_address, and latlon Columns
The Location column contained strings that housed multiple pieces of geographic information (e.g., '6000 CHINQUAPIN PKWY\nBaltimore, MD\n(39.365093, -76.59764)') and thus required two cleaning steps: writing and applying functions to split the strings into their constituent parts and save those parts as new columns (latitude, longitude, street_address, latlon) in the DataFrame.

In [7]:
def latitude(location):
    """Ingest Location and return latitude"""
    lat = float('NaN')
    if type(location) == str:
        if ((len(location.split('\n')) > 2) and (location.split('\n')[2] != '')):
            lat = float(location.split('\n')[2].split('(')[1].split(',')[0])
    return lat


def longitude(location):
    """Ingest Location and return longitude"""
    lon = float('NaN')
    if type(location) == str:
        if ((len(location.split('\n')) > 2) and (location.split('\n')[2] != '')):
            lon = float(location.split('\n')[2].split('(')[1].split(',')[1].split(')')[0])       
    return lon


def street_address(location):
    """Ingest Location and return street address"""
    address = ''
    if type(location) == str and len(location.split('\n')) >=1 :
        address = location.split('\n')[0]       
    return address


def latlon(location):
    """Ingest Location and return tuple of latitude, longitude"""
    latlon = ''
    if type(location) == str and ((len(location.split('\n')) > 2) and (location.split('\n')[2] != '')):
            latlon = (location.split('\n')[2])
    return latlon


df['latitude']       = df['Location'].apply(latitude)
df['longitude']      = df['Location'].apply(longitude)
df['street_address'] = df['Location'].apply(street_address)
df['latlon']         = df['Location'].apply(latlon)

### V. Set the DateTimeIndex
Next, we set the ViolDate column to be the DateTimeIndex of the DataFrame.

In [8]:
df_full.index = df_full['ViolDate']
df.drop('ViolDate', axis=1, inplace=True)

### VI. Handle Missing Data
Once the dataset was clean, the next step in data wrangling was to deal with missing data. 

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753387 entries, 5 to 1345052
Data columns (total 20 columns):
Citation          753387 non-null int64
Tag               753260 non-null object
State             753387 non-null object
Make              752981 non-null object
Address           753387 non-null object
ViolCode          753387 non-null float64
Description       753387 non-null object
ViolFine          753387 non-null float64
Balance           753387 non-null float64
OpenFine          753387 non-null float64
OpenPenalty       753387 non-null float64
Location          743041 non-null object
year              753387 non-null int64
month             753387 non-null int64
day               753387 non-null int64
hour              753387 non-null int64
latitude          529579 non-null float64
longitude         529579 non-null float64
street_address    753387 non-null object
latlon            753387 non-null object
dtypes: float64(7), int64(5), object(8)
memory usage: 140.7+ MB


#### A) Drop Nulls

As shown above, the DataFrame now consists of 20 columns and 753,387 rows. 17 of the columns (all except latitude, longitude, and latlon) contain <= 1.5% missing data. Given this low percentage of missing data, we found it acceptable to simply drop the observations that contained missing values in one or more of these 17 columns. 

In [10]:
df.dropna(axis=0, how='any', subset=['Citation', 'Tag', 'State', 'Make', 'Address', 'ViolCode', 'Description', 'ViolFine', 'Balance', 'OpenFine', 'OpenPenalty', 'Location', 'year', 'month', 'day', 'hour', 'street_address'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 742513 entries, 5 to 1345052
Data columns (total 20 columns):
Citation          742513 non-null int64
Tag               742513 non-null object
State             742513 non-null object
Make              742513 non-null object
Address           742513 non-null object
ViolCode          742513 non-null float64
Description       742513 non-null object
ViolFine          742513 non-null float64
Balance           742513 non-null float64
OpenFine          742513 non-null float64
OpenPenalty       742513 non-null float64
Location          742513 non-null object
year              742513 non-null int64
month             742513 non-null int64
day               742513 non-null int64
hour              742513 non-null int64
latitude          529161 non-null float64
longitude         529161 non-null float64
street_address    742513 non-null object
latlon            742513 non-null object
dtypes: float64(7), int64(5), object(8)
memory usage: 119.0+ MB


This left us with 742,513 observations or 98.6% of the original dataset. 

#### B) Dealing with missing data in the latitude, longitude, and latlon columns

In [11]:
df['latitude'].notnull().sum()/df['Citation'].count()*100
df['longitude'].notnull().sum()/df['Citation'].count()*100
df['Citation'][df['latlon'] != ''].count()/df['Citation'].count()*100

71.266226988618385

71.266226988618385

71.266226988618385

Of the 742,513 observations, only 71.3% contained values in the latlon, latitude, and longitude columns. We attempted to remedy this by creating a dictionary with key-value pairs of Address:latlon and using the dictionary to populate a new_latlon column. 

In [12]:
address_latlon = zip(df['Address'], df['latlon'])
address_dict = dict(address_latlon)
df['new_latlon'] = df['Address'].map(address_dict)
df['Citation'][df['new_latlon'] != ''].count()/df['Citation'].count()

0.71585547997139443

This resulted in a very small success rate, however: 0.3 % increase in poplulated observations. Consequently, we must be reserved in our interpretation of any geographic trends.

### VII. Reorder and rename columns
The final data wrangling step requried was to reorder and rename the columns to convenient names. 

In [13]:
df = df[['Citation','Tag', 'State', 'Make', 'year', 'month', 'day', 'hour', 'ViolCode', 'Description', 'Address','new_latlon', 'latitude', 'longitude',  'ViolFine', 'Balance', 'OpenFine', 'OpenPenalty']]
df.columns =['citation', 'tag', 'state', 'make', 'year', 'month', 'day', 'hour', 'code', 'description', 'address', 'latlon', 'latitude', 'longitude', 'fine', 'balance', 'open_fine', 'open_penalty']